In [74]:
%reset -f
import pandas as pd
import requests
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [75]:
# insert gw of interest
current_gw = 5

In [76]:
# api endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
fixtures = 'https://fantasy.premierleague.com/api/fixtures/'

In [77]:
# making requests
response = requests.get(url)
fixtures = requests.get(fixtures).json()

# check status
if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

#print(json.dumps(fixtures, indent=4, sort_keys=True))

The API request was successful.


In [78]:
# mapping teams
teams = response.json()['teams']
team_mapping = {}

for team in teams:
    team_mapping[team['id']] = team['name']

In [79]:
# team difficulty - start of season
team_difficulty = {
    "ARS": 6,
    "AVL": 4, 
    "BOU": 2,
    "BRE": 4,
    "BRI": 5,
    "BUR": 1,
    "CHE": 4,
    "CRY": 3,
    "EVE": 2,
    "FUL": 3, 
    "LIV": 6,
    "LUT": 1,
    "MCI": 7,
    "MUN": 5,
    "NEW": 5,
    "NFO": 3,
    "SHU": 1,
    "TOT": 5,
    "WHU": 4, 
    "WOL": 2
}

In [80]:
name_mapping = {
    "Arsenal": "ARS",
    "Aston Villa": "AVL", 
    "Bournemouth": "BOU",
    "Brentford": "BRE",
    "Brighton": "BRI",
    'Burnley': "BUR",
    "Chelsea": "CHE",
    "Crystal Palace": "CRY",
    "Everton": "EVE",
    "Fulham": "FUL", 
    "Liverpool": "LIV",
    "Luton": "LUT",
    "Man City": "MCI",
    "Man Utd": "MUN",
    "Newcastle": "NEW",
    "Nott'm Forest": "NFO",
    "Sheffield Utd": "SHU",
    "Spurs": "TOT",
    "West Ham": "WHU", 
    "Wolves": "WOL"
}

In [81]:
freq = defaultdict(int)
for v in team_difficulty.values():
    freq[v] += 1
    
df = pd.DataFrame.from_dict(freq, orient='index', columns=['Count'])
df.index.name = 'Difficulty'
df = df.sort_index(ascending=False)

df

,Count
Difficulty,
7,1
6,2
5,4
4,4
3,3
2,3
1,3


In [82]:
data = []

for match in fixtures:
    if 'event' not in match or match['event'] is None:
        continue
    elif match['event'] >= current_gw and match['event'] <= (current_gw + 10):
        #print(f"GW {match['event']}: Home: {team_mapping[match['team_h']]} Away:{team_mapping[match['team_a']]}")
        # Extract teams  
        home_team = team_mapping[match['team_h']]
        away_team = team_mapping[match['team_a']]
    
        # append to list as dict
        data.append({
          'event': match['event'],
          'Team': home_team,
          'Team2': away_team
        })

        data.append({
          'event': match['event'],
          'Team': away_team,
          'Team2': home_team
        })

df = pd.DataFrame(data)

# shorten team names
df["Team"] = df["Team"].map(lambda x: name_mapping[x])
df["Team2"] = df["Team2"].map(lambda x: name_mapping[x])

# group by 'event' and 'team1', then concatenate 'team2' values
df['Team2'] = df.groupby(['event', 'Team'])['Team2'].transform(lambda x: ', '.join(x))

# drop duplicates and reset index
df.drop_duplicates(subset=['event', 'Team'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [83]:
# pivot to wide format
fdr = df.pivot(index='Team', columns='event', values='Team2')

# rename columns 
fdr.columns = [f'GW{col}' for col in fdr.columns]
fdr.fillna('-', inplace = True)

fdr

,GW5,GW6,GW7,GW8,GW9,GW10,GW11,GW12,GW13,GW14,GW15
Team,,,,,,,,,,,
ARS,EVE,TOT,BOU,MCI,CHE,SHU,NEW,BUR,BRE,WOL,LUT
AVL,CRY,CHE,BRI,WOL,WHU,LUT,NFO,FUL,TOT,BOU,MCI
BOU,CHE,BRI,ARS,EVE,WOL,BUR,MCI,NEW,SHU,AVL,CRY
BRE,NEW,EVE,NFO,MUN,BUR,CHE,WHU,LIV,ARS,LUT,BRI
BRI,MUN,BOU,AVL,LIV,MCI,FUL,EVE,SHU,NFO,CHE,BRE
BUR,NFO,MUN,...,CHE,BRE,BOU,CRY,ARS,WHU,SHU,WOL
CHE,BOU,AVL,FUL,BUR,ARS,BRE,TOT,MCI,NEW,BRI,MUN
CRY,AVL,FUL,MUN,NFO,NEW,TOT,BUR,EVE,LUT,WHU,BOU
EVE,ARS,BRE,LUT,BOU,LIV,WHU,BRI,CRY,MUN,NFO,NEW


In [84]:
def gradient_color(val):
    if val in team_difficulty:
        difficulty = team_difficulty[val]
        norm = plt.Normalize(min(team_difficulty.values()), max(team_difficulty.values()))
        cmap = plt.cm.RdYlGn_r 
        color = mcolors.to_hex(cmap(norm(difficulty)))
        return f"background-color: {color}"
    return ""

styled_fdr = fdr.style.applymap(gradient_color)
styled_fdr

,GW5,GW6,GW7,GW8,GW9,GW10,GW11,GW12,GW13,GW14,GW15
Team,,,,,,,,,,,
ARS,EVE,TOT,BOU,MCI,CHE,SHU,NEW,BUR,BRE,WOL,LUT
AVL,CRY,CHE,BRI,WOL,WHU,LUT,NFO,FUL,TOT,BOU,MCI
BOU,CHE,BRI,ARS,EVE,WOL,BUR,MCI,NEW,SHU,AVL,CRY
BRE,NEW,EVE,NFO,MUN,BUR,CHE,WHU,LIV,ARS,LUT,BRI
BRI,MUN,BOU,AVL,LIV,MCI,FUL,EVE,SHU,NFO,CHE,BRE
BUR,NFO,MUN,"NEW, LUT",CHE,BRE,BOU,CRY,ARS,WHU,SHU,WOL
CHE,BOU,AVL,FUL,BUR,ARS,BRE,TOT,MCI,NEW,BRI,MUN
CRY,AVL,FUL,MUN,NFO,NEW,TOT,BUR,EVE,LUT,WHU,BOU
EVE,ARS,BRE,LUT,BOU,LIV,WHU,BRI,CRY,MUN,NFO,NEW
